## 텍스트 사전 준비 작업 (텍스트 전처리) - 텍스트 정규화

### 클렌징
분석에 불필요한 문자 기호 등을 사전에 제거하는 작업

### 텍스트 토큰화
문장 토큰화 (sentence tokenization): 문장의 마침표, 개행문자 등 문장의 마지막을 뜻하는 기호에 따라 분리
NLTK의 sent_tokenize

In [2]:
from nltk import sent_tokenize
import nltk
nltk.download("punkt")

text_sample = "The matrix is everywhere its all around us, here even in this room. You can see it out your window or on your television. You feel it when you go to work, or go to church or pay your taxes."
sentences = sent_tokenize(text_sample)

[nltk_data] Downloading package punkt to /Users/jinjae/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
print(type(sentences), len(sentences))
print(sentences)

<class 'list'> 3
['The matrix is everywhere its all around us, here even in this room.', 'You can see it out your window or on your television.', 'You feel it when you go to work, or go to church or pay your taxes.']


단어 토큰화 (word tokenization): 문장을 단어로 토큰화

In [4]:
from nltk import word_tokenize

sentence = "The matrix is everywhere its all around us, here even in this room."
words = word_tokenize(sentence)
print(type(words), len(words))
print(words)

<class 'list'> 15
['The', 'matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.']


In [5]:
from nltk import word_tokenize, sent_tokenize

def tokenize_text(text):
    sentences = sent_tokenize(text)
    word_tokens = [word_tokenize(sentence) for sentence in sentences]
    return word_tokens

word_tokens = tokenize_text(text_sample)
print(type(word_tokens), len(word_tokens))
print(word_tokens)

<class 'list'> 3
[['The', 'matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.'], ['You', 'can', 'see', 'it', 'out', 'your', 'window', 'or', 'on', 'your', 'television', '.'], ['You', 'feel', 'it', 'when', 'you', 'go', 'to', 'work', ',', 'or', 'go', 'to', 'church', 'or', 'pay', 'your', 'taxes', '.']]


### 스톱 워드 제거

분석에 큰 의미가 없는 단어 제거

In [6]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jinjae/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
print("English stop words counts:", len(nltk.corpus.stopwords.words("english")))
print(nltk.corpus.stopwords.words("English")[:30])

English stop words counts: 179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself']


In [8]:
import nltk

stopwords = nltk.corpus.stopwords.words("english")
all_tokens = []

# previous sample
for sentence in word_tokens:
    filtered_words = []
    for word in sentence:
        # to lowercase
        word = word.lower()
        if word not in stopwords:
            filtered_words.append(word)
    all_tokens.append(filtered_words)

print(all_tokens)

[['matrix', 'everywhere', 'around', 'us', ',', 'even', 'room', '.'], ['see', 'window', 'television', '.'], ['feel', 'go', 'work', ',', 'go', 'church', 'pay', 'taxes', '.']]


## Stemming 과 Lemmatization

과거/현재, 3인칭 단수 여부, 진행형 등에 따라 단어 변화 → 단어의 원형을 찾는 작업

Lemmatization은 Stemming 보다 정교하며 의미론적인 기반에서 단어의 원형을 찾음

Stemming

- 일부 철자가 훼손된 어근 단어 추출
- NLTK: Porter, Lancaster, Snowball Stemmer

In [9]:
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()

print(stemmer.stem("working"), stemmer.stem("works"), stemmer.stem("worked"))
print(stemmer.stem("amusing"), stemmer.stem("amuses"), stemmer.stem("amused"))
print(stemmer.stem("happier"), stemmer.stem("happiest"))
print(stemmer.stem("fancier"), stemmer.stem("fanciest"))

work work work
amus amus amus
happy happiest
fant fanciest


Lemmatization

- 문법적인 요소와 의미적인 부분을 감안 (시간이 더 걸림)
- NLTK: WordNetLemmatizer

In [11]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download("omw-1.4")
nltk.download("wordnet")

lemma = WordNetLemmatizer()
print(lemma.lemmatize("amusing", 'v'), lemma.lemmatize("amuses", 'v'), lemma.lemmatize("amused", 'v'))
print(lemma.lemmatize("happier", 'a'), lemma.lemmatize("happiest", 'a'))
print(lemma.lemmatize("fancier", 'a'), lemma.lemmatize("fanciest", 'a'))

[nltk_data] Downloading package omw-1.4 to /Users/jinjae/nltk_data...
[nltk_data] Downloading package wordnet to /Users/jinjae/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


amuse amuse amuse
happy happy
fancy fancy


## BOW (Bag of Words)

문맥 순서를 무시하고 모든 단어에 대해 빈도 값을 부여해 피처 값 추출

1. 문장 내 모든 단어에서 중복을 제거하고 인덱스 부여
2. 개별 문장에서 해당 단어가 나타내는 횟수를 단어에 기재

단점

- 문맥 의미 반영 부족: 단어의 문맥적인 해석을 처리하지 못함
- 희소 행렬 문제: 대규모 칼럼 중 대부분의 값이 0으로 채워지는 행렬의 발생 (희소 행렬은 알고리즘 수행 시간과 예측 성능을 떨어트림)

### BOW 피처 벡터화

텍스트와 같은 데이터는 알고리즘에 바로 입력할 수 없음 → 벡터 값으로 변환

- 카운트 기반 벡터화
    - 각 문서에서 해당 단어가 나타나는 횟수를 부여
- TF-IDF 기반 벡터화
    - 개별 문서에서 자주 나타나는 단어에 높은 가중치 부여, 모든 문서에서 전반적으로 나타나는 단어에 패널티 부여

문서마다 텍스트가 길고 개수가 많은 경우 TF-IDF가 더 좋은 예측 성능 보장

## 사이킷런 CountVectorizer, TfidfVectorizer

CountVectorizer 파라미터

- max_df: 너무 높은 빈도수를 가진 단어 피처 제외
- min_df: 너무 낮은 빈도수를 가진 단어 피처 제외
- max_features: 가장 높은 것부터 개수를 제한
- stop_words: ‘english’로 지정하면 영어에서 스톱 워드 제외
- n_gram_range: n_gram 범위 설정
- analyzer: 피처 추출을 수행한 단위 지정
- token_pattern: 토큰화를 수행하는 정규 표현식 패턴 지정
- tokenizer: 토큰화를 별도 커스텀 함수로 이용할 때 적용

수행 과정

1. 소문자 변경 등 전처리
2. n_gram_range 반영 단어 토큰화
3. 텍스트 정규화 수행
4. 토큰화된 단어 피처로 추출 및 벡터화

TfidfVectorizer 또한 거의 유사

## BOW 벡터화를 위한 희소 행렬

CountVectorizer / TfidfVectorizer

텍스트를 피처 단위로 벡터화해 변환 → CSR 형태의 희소 행렬 반환

대규모 행렬(수만 수십 개) 중에서 각 문서가 가지는 단어 수는 제한적이므로 행렬 값은 대부분 0 ⇒ 희소 행렬

메모리 공간 많이 필요, 데이터 엑세스 시간 다소 소모

COO, CSR 형식으로 변환

## 희소 행렬 - COO 형식

COO (Coordinate) 형식: 0이 아닌 데이터만 별도의 데이터 배열에 저장하고 행과 열의 위치를 별도의 배열로 저장

Scipy 이용

In [12]:
import numpy as np

dense = np.array([ [3, 0, 1], [0, 2, 0] ])

In [13]:
from scipy import sparse

data = np.array([3, 1, 2])

row_pos = np.array([0, 0, 1])
col_pos = np.array([0, 2, 1])

sparse_coo = sparse.coo_matrix((data, (row_pos, col_pos)))

In [14]:
sparse_coo.toarray()

array([[3, 0, 1],
       [0, 2, 0]])

## 희소 행렬 - CSR 형식

행 위치 배열의 고유한 값의 시작 위치만 표기 + 행 위치 배열의 총 항목 개수

In [18]:
from scipy import sparse

dense2 = np.array([[0, 0, 1, 0, 0, 5],
                  [1, 4, 0, 3, 2, 5],
                  [0, 6, 0, 3, 0, 0],
                  [2, 0, 0, 0, 0, 0],
                  [0, 0, 0, 7, 0, 8],
                  [1, 0, 0, 0, 0, 0]])

data2 = np.array([1, 5, 1, 4, 3, 2, 5, 6, 3, 2, 7, 8, 1])

row_pos = np.array([0, 0, 1, 1, 1, 1, 1, 2, 2, 3, 4, 4, 5])
col_pos = np.array([2, 5, 0, 1, 3, 4, 5, 1, 3, 0, 3, 5, 0])

sparse_coo = sparse.coo_matrix((data2, (row_pos, col_pos)))

row_pos_ind = np.array([0, 2, 7, 9, 10, 12, 13])

sparse_csr = sparse.csr_matrix((data2, col_pos, row_pos_ind))

print(sparse_coo.toarray())
print(sparse_csr.toarray())
assert(sparse_coo.toarray().all() == sparse_csr.toarray().all())

[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]
[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]
